In [ ]:
# run in cellpose_2 conda environment in the alien computer
import os
import time
import numpy as np
import tifffile

from skimage.morphology import white_tophat, black_tophat, disk
from skimage import io, measure, util, morphology
from scipy import ndimage
from cellpose import models, io
from tqdm import tqdm

In [ ]:
# read image
file_name = ""
img = io.imread(file_name)

print(img.shape)
print(img.dtype)

In [ ]:
# segment nuclei
seg_img = img[3, :, :]

## cellpose parameters
diameter = 30
flow_threshold = 0
mask_threshold = -6
do_3D = True
stitch_threshold = 0
gpu = True

## run cellpose in 3D mode
t1 = time.time()
model = models.Cellpose(gpu = gpu, model_type = 'cyto')
masks, flows, styles, diams = model.eval(
    seg_img, 
    channels = [0,0], 
    diameter = diameter, 
    do_3D = do_3D, 
    flow_threshold = flow_threshold, 
    cellprob_threshold = mask_threshold
    )
t2 = time.time()
time_elapsed = (t2 - t1) / 60
print('Time spent in current image: %s minutes' % round(time_elapsed,1))

## save mask image
mask_output_path = f"{os.path.basename(file_name)[:-4]}_dia-{diameter}_3D-{do_3D}_stitch-{stitch_threshold}.tif"
io.imsave(mask_output_path, masks)


In [ ]:
# measure nuclear intensities 
